In [1]:
import pandas as pd
import difflib
import networkx as nx
import matplotlib.pyplot as plt
from rapidfuzz import fuzz

In [2]:
ruta_archivo = "Dataset_Form_Abril.xlsx"
df = pd.read_excel(ruta_archivo)

In [3]:
def normalizar(nombre):
    if pd.isna(nombre):
        return ""
    return " ".join(nombre.strip().upper().split())

In [4]:
colabs = {}
puestos = {}
for _, row in df.iterrows():
    nombre = normalizar(row["NOMBRES_COLABORADORES"])
    cedula = str(row["CED"]).strip()
    puesto = str(row["PUESTO"]).strip().upper()
    colabs[nombre] = cedula
    puestos[nombre] = puesto

In [5]:
relaciones=[]

In [ ]:
for _, row in df.iterrows():
    declarante = normalizar(row["NOMBRES_COLABORADORES"])
    cedula_decl = str(row["CED"]).strip()
    puesto = str(row["PUESTO"]).strip().upper()
    
    for i in range(1, 5):
        pariente = normalizar(row.get(f"NOMBRES PARIENTE {i}", ""))
        grado = str(row.get(f"GRADO DE PARENTESCO {i}", "")).strip().upper()
        
        
        if pariente and pariente != "NAN":
            relaciones.append({
                "DECLARANTE": declarante,
                "CED_DECLARANTE": cedula_decl,
                "PUESTO_DECLARANTE": puesto,
                "PARIENTE": pariente,
                "GRADO": grado
            })

In [7]:
# Buscar inconsistencias
inconsistencias = []
for r in relaciones:
    a = r["DECLARANTE"]
    b = r["PARIENTE"]
    grado = r["GRADO"]
    
    posibles = difflib.get_close_matches(b, colabs.keys(), n=1, cutoff=0.85)
    if posibles:
        b_real = posibles[0]
        reciproco = any(
            (r2["DECLARANTE"] == b_real and difflib.SequenceMatcher(None, r2["PARIENTE"], a).ratio() > 0.85)
            for r2 in relaciones
        )
        if not reciproco:
            # Evaluar riesgo por grado de parentesco
            riesgo_alto = {"PADRE/MADRE", "HERMANO/A", "HIJO/A", "CONYUGE","ABUELO/A"}
            riesgo_medio = {"SOBRINO/A", "TIO/A","PRIMO/A"}
            riesgo_bajo = {"SUEGRO/A", "YERNO/NUERA","CUNADO/A"}

            if grado in riesgo_alto:
                riesgo = "ALTO"
            elif grado in riesgo_medio:
                riesgo = "MEDIO"
            elif grado in riesgo_bajo:
                riesgo = "BAJO"
            else:
                riesgo = "NO DEFINIDO"

            inconsistencias.append({
                "COLABORADOR A (DECLARANTE)": a,
                "CÉDULA A": r["CED_DECLARANTE"],
                "COLABORADOR B (PARIENTE)": b_real,
                "CÉDULA B": colabs[b_real],
                "GRADO DE PARENTESCO": grado,
                "RIESGO": riesgo,
                "COMENTARIO": "NO RECIPROCIDAD"
            })

In [8]:
df_inconsistencias = pd.DataFrame(inconsistencias)
df_inconsistencias.to_excel("inconsistencias_con_riesgo.xlsx", index=False)
print("Archivo 'inconsistencias_con_riesgo.xlsx' generado.")


Archivo 'inconsistencias_con_riesgo.xlsx' generado.


# GRUPOS DE FAMILIAS

In [11]:
nombres_unicos = set()

for _, row in df.iterrows():
    nombres_unicos.add(normalizar(row["NOMBRES_COLABORADORES"]))
    for i in range(1, 5):
        nombres_unicos.add(normalizar(row.get(f"NOMBRES PARIENTE {i}", "")))
        for i in range(1,9):
            nombres_unicos.add(normalizar(row.get(f"NOMBRES PARIENTE ADICIONAL {i}","")))

nombres_unicos = list(filter(None, nombres_unicos))

umbral_similitud = 80
grupos_similares = []

while nombres_unicos:
    base = nombres_unicos.pop(0)
    grupo = [base]
    similares = []

    for otro in nombres_unicos:
        if fuzz.token_sort_ratio(base, otro) >= umbral_similitud:
            grupo.append(otro)
            similares.append(otro)

    for s in similares:
        nombres_unicos.remove(s)

    grupos_similares.append(grupo)

# Crear un mapa de reemplazo (usar el nombre más corto del grupo como estándar)
mapa_normalizado = {}
for grupo in grupos_similares:
    estandar = min(grupo, key=len)
    for nombre in grupo:
        mapa_normalizado[nombre] = estandar

G = nx.Graph()
nombres_cedulas = {}

for _, row in df.iterrows():
    declarante = normalizar(row["NOMBRES_COLABORADORES"])
    declarante = mapa_normalizado.get(declarante, declarante)
    ced_declarante = str(row["CED"]).strip()
    nombres_cedulas[declarante] = ced_declarante

    for i in range(1, 5):
        pariente = normalizar(row.get(f"NOMBRES PARIENTE {i}", ""))
        pariente = mapa_normalizado.get(pariente, pariente)
        if pariente and pariente != "NAN":
            G.add_edge(declarante, pariente)

# Familias
familias = list(nx.connected_components(G))

datos_familias = []
for i, familia in enumerate(familias, start=1):
    identificador = f"FAM{i:03d}"
    nombres = list(familia)
    cedulas = [nombres_cedulas.get(n, "SIN_CED") for n in nombres]
    num_integrantes = len(nombres)

    if num_integrantes <= 2:
        riesgo = "BAJO"
    elif 3 <= num_integrantes <= 4:
        riesgo = "MEDIO"
    else:
        riesgo = "ALTO"

    datos_familias.append({
        "Identificador de familiar": identificador,
        "SEC": "-".join(cedulas),
        "Familia": ", ".join(nombres),
        "Número de integrantes": num_integrantes,
        "Riesgo": riesgo
    })


In [12]:
# Guardar
df_familias = pd.DataFrame(datos_familias)
df_familias.to_excel("familias_identificadas_filtradas.xlsx", index=False)
print("Archivo 'familias_identificadas_filtradas.xlsx' generado correctamente.")

Archivo 'familias_identificadas_filtradas.xlsx' generado correctamente.
